In [436]:
#Importing the required libraries.
import pandas as pd #for dataframes and reading csv files.
import numpy as np # for math operations(sqrt)
import matplotlib.pyplot as plt #for plotting the graphs



In [437]:
file_day_od="data/day_od.csv" #directory of the file(file_path)
#csv (comma seperated values, delimiter is comma)
data_day_od=pd.read_csv(file_day_od,sep=",",encoding='utf-16', header=0)
                       
#quoting=2,quotechar='"',escapechar='\r' )
#comment: explicit header=0 to replace column names;  quoting=2 "Non-Numerical"; es

#seperate the columns by name
headerName=["MONTH","DOW","ORIGIN","DESTINATION","CUST_CLASS","COD_COUNTRY","COD_PRO","PRO_COM","FLOW"]
data_day_od.columns=headerName
day_od=pd.DataFrame(data_day_od)#dataFrame of the file(can be read more easily than csv)

print(data_day_od)

# Getting DataFrame for distinct_users_day.csv
file_distinct_users_day="data/distinct_users_day.csv"
data_distinct_users_day=pd.read_csv(file_distinct_users_day,sep=",",encoding="latin-1")
                                    #,quoting=3,escapechar="\"",encoding="latin-1")
distinct_users_day=pd.DataFrame(data_distinct_users_day)
print(distinct_users_day)
# Getting DataFrame for codici_istat_comune.csv
file_codici_istat_comune="data/codici_istat_comune.csv"
data_codici_istat_comune=pd.read_csv(file_codici_istat_comune,sep=",",delimiter=',',header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_comune=pd.DataFrame(data_codici_istat_comune)
#codici_istat_comune

# Getting DataFrame for codici_istat_provincia.csv
file_codici_istat_provincia="data/codici_istat_provincia.csv"
data_codici_istat_provincia=pd.read_csv(file_codici_istat_provincia,sep=",",header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_provincia=pd.DataFrame(data_codici_istat_provincia)
#codici_istat_provincia

# Getting DataFrame for codici_nazioni.csv
import csv
file_codici_nazioni="data/codici_nazioni.csv"
data_codici_nazioni=pd.read_csv(file_codici_nazioni,delimiter=',',header=0,quoting=2, 
                                error_bad_lines=True,encoding="utf-16",quotechar = '"',escapechar='\r')
codici_nazioni=pd.DataFrame(data_codici_nazioni)
 #codici_nazioni[108:112]

           MONTH       DOW  ORIGIN  DESTINATION CUST_CLASS  COD_COUNTRY  \
0          Marzo  Domenica     108          300    visitor          222   
1         Maggio    Lunedì     300          101    visitor          222   
2       Febbraio    Sabato     108          207    visitor          222   
3         Aprile   Venerdì     109          121   resident          222   
4         Maggio   Giovedì     300          120    visitor          222   
...          ...       ...     ...          ...        ...          ...   
193496  Febbraio   Martedì     205          120      Altro         -999   
193497     Marzo   Giovedì     214          115      Altro         -999   
193498  Febbraio   Giovedì     124          121      Altro         -999   
193499    Aprile  Domenica     110          300    visitor          222   
193500    Aprile    Lunedì     124          300    visitor          222   

        COD_PRO  PRO_COM  FLOW  
0          28.0   -999.0   493  
1          93.0   -999.0    58  


## POINT 4

 Study of the visitors' fluxes: let's assume to be asked to provide indications of how to invest resources to improve the mobility toward Padova. Defined the three main directions of visitors and commuter getting to Padova by the Italian highways (sud: A13 toward Bologna-Roma / west: A4 toward Milano-Torino / north-east: A4 toward Venice-Trieste), evaluate which of the three directions has to be prioritized:
 1. Consider a simplified case involving only the mid-range mobility, based on the number of visitors/commuters from the nearby regions only
 * Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week
 * _[OPEN]_ Use the data available to provide what you believe is the best possible answer 

## 1 

In this first part to study the visitors's fluxes from the nearby regions only we've selected one region for each of the different directions toward Padova


-Lombardia at west, associated with the A4 toward Milano-Torino <br>
-Emilia Romagna at south, assiciated with the A13 toward Bologna-Roma  <br>
-Friuli Venezia Giulia at east, assiciated with the A4 toward Venice-Trieste. <br> 
 

The first step is to obtain all the data for the selected regions: we first need to find region's cod (info_region func and d\[reg\[i\]\] dataframe) in order to extract from the dataset data_od_od the information for our selected regions. (data_reg\[reg\[i\]\] dataset, contains all provinces)

We can then  calculate the total fluxes (flow_region function)  in order to value which directions shoud be proritized 

In [443]:
#DATA FOR THE REGIONS OF INTEREST
#For the regions of interest insert the region capital names and the region names
cap=['Milano', 'Bologna', 'Trieste',]
reg=['Lombardia', 'Emilia Romagna', 'Friuli Venezia Giulia']

#Function to obtain the info of the regions of interest 
def info_regions(codici_istat_provincia, capoluogo):
    cod_reg=codici_istat_provincia[codici_istat_provincia['PROVINCIA'] == capoluogo]['COD_REG'].values[0] 
    info_reg=codici_istat_provincia[codici_istat_provincia['COD_REG'] == cod_reg]
    return info_reg

#Create a dictionary of dataframes for the different regions (cod info only)
d = {} 
cod_int=[]
for c in cap:
    d[reg[cap.index(c)]] = pd.DataFrame(info_regions(codici_istat_provincia,c))
    #dataframe d[key] with key=reg[i], data: |COD_REG|COD_PRO|PROVINCIA|PROV_SIGLA
    

#Returns a dataset with all the info (data_day_od) of the regions of interest
def data_regions(data_day_od, cod_all_prov):
    data_reg=data_day_od[data_day_od['COD_PRO'].isin(cod_all_prov)]
    return data_reg

#Define a dictionary with the dataframe for the different regions with all the info from data_day_od
#data_reg[reg[i]]  MONTH|DOW|ORIGIN|DESTINATION|CUST_CLASS|COD_CONTRY|COD_PRO|PRO_COM|FLOW
data_reg = {} 
for i in range(len(reg)):
    data_reg[reg[i]] = pd.DataFrame(data_regions(data_day_od,d[reg[i]]['COD_PRO'].values[:]))
    

##STUDY OF THE FLUXES OF NEARBY REGIONS TO FIND WHICH OF THE THREE DIRECTIONS HAS TO BE PRIORITIZE

#Function to calculate the total flow of the regions of interest 
def flow_regions(info_data, cod_all_prov):
    flow_sum=info_data.groupby(info_data['COD_PRO'].isin(cod_all_prov))['FLOW'].sum()
    flow_sum=flow_sum.reset_index()
    return flow_sum['FLOW'][flow_sum['COD_PRO'] == True][0]
    
#Create a dictionary for the the total flux for the different regions 
d_flow = {}
for i in range(len(reg)):
    d_flow[reg[i]] = flow_regions(data_reg[reg[i]], d[reg[i]]['COD_PRO'].values[:])
    
    
#Dataframe with the total flux for the regions of interest, flow: REGION|FLOW|
flow= pd.DataFrame(d_flow.items(), columns=['REGION', 'FLOW']).sort_values(by='FLOW', ascending=False)


#Display the results
import plotly.graph_objs as go
import plotly.figure_factory as ff


table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]

# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(flow, height_constant=80, colorscale=table_color) # font_colors=font)#colorscale

color_dict = ['#b32d00', '#fb9804' ,'#b5de2b']
fig.add_trace(go.Bar(x = flow["REGION"], y = flow["FLOW"],marker_color=color_dict,name='Flow',xaxis='x2', yaxis='y2',))
    
fig.update_layout(title_text = 'Total flow for the nearby regions',margin = {'t':60, 'b':80, },
    xaxis = {'domain': [0, .4]},
    xaxis2 = {'domain': [0.5, 1.], 'title':'Region'},
    yaxis2 = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'}
)

fig.show()

    

As we can see from this plot the region with the highest flux is "Lombardia", followed by "Emilia Romagna" and "Friuli Venezia Giulia". Of the three directions, the Italian highways A4, torward Milano-Torino, should be therefore proritized. 


## 2

In [439]:
# 4.2: STUDY OF THE FLUXES OF THE MOSTLY CONTRIBUTING PROVINCES BASED ON THE DAY OF THE WEEK 

#SELECTION OF THE PROVINCES THAT ARE MOSTLY CONTRIBUTING BASED ON THE FLOW 

#Functions that finds the mostly contributing provinces for each region:
#Returns the total flux for each province and the code of the selected provinces
def flow_province(data_regions):
    flow_province=data_regions.groupby('COD_PRO').FLOW.sum()
    flow_sorted=flow_province.sort_values(ascending=False)
    #flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index()
    flow_selected=flow_sorted.head(n=3).reset_index()  #We choose the first 3 with the highest flux
    cod_pro_selected=flow_selected['COD_PRO'].values[:]
    return flow_selected, cod_pro_selected

d_flow_province = {} #for function output
cod_pro_selected={} #for function output
flow_pro={} #to built a dictionary with a flow dataframe for the different regions
pro_selected={}  #to built a dictionary with the code of the selected provinces

for i in range(len(reg)): 
    d_flow_province, cod_pro_selected = flow_province(data_reg[reg[i]]) #function returns two different outputs
    flow_pro[reg[i]]= d_flow_province
    pro_selected[reg[i]]=cod_pro_selected
    

#Associating each province code the province name
#flow of the province for a region flow_pro[reg[i]]: 'COD PRO', 'NAME', 'FLOW'

for i in range(len(reg)):
    name_l=[]
    for cod in flow_pro[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] 
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_l.append(name)
    flow_pro[reg[i]].insert(1, "NAME", name_l)

#Display the results
import plotly.graph_objs as go
import plotly.figure_factory as ff

result=pd.concat(flow_pro.values(), axis=1)

fig5 = go.Figure()
color_dict = ['#b32d00', '#fb9804' ,'#b5de2b']


for i in range(len(reg)):  
        fig5.add_trace(go.Bar(x = flow_pro[reg[i]]["NAME"], y = flow_pro[reg[i]]["FLOW"], name=reg[i], marker_color=color_dict[i], text="Automatic Labels Based on Data Frame Column Names"))
        fig5.update_layout(barmode='relative')
       # fig5.add_trace(go.Bar(x = flow_pro[reg[i]]["NAME"], y = flow_pro[reg[i]]["FLOW"],name =reg[i]))
        fig5.update_layout(title={'text': "Flow for the mostly contributing provinces",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Province'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})

fig5.show()



table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
    
for i in range(len(reg)):  
    fig = ff.create_table(flow_pro[reg[i]],colorscale=table_color)
    fig.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
    fig.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .7]}, yaxis1 = {'domain': [0., 1]})
    fig.show()
    

In [440]:
#STUDY OF THE FLUXES BASED ON THE DAY OF THE WEEK

#Function that returns a dataframe with all the info (data_day_od solo) for the selected provinces for each region
def data_selected_province(data_day_od,cod_pro_selected):
    data_province_selected=data_day_od[data_day_od['COD_PRO'].isin(cod_pro_selected)] 
    return data_province_selected

#Define a dictionary with a dataframe for the different regions, containg all the info of the selected provinces 
d_selected_province = {} 
for i in range(len(reg)):
    d_selected_province[reg[i]] = pd.DataFrame(data_selected_province(data_day_od,flow_pro[reg[i]]['COD_PRO'].values[:]))
    
#print(d_selected_province)

#Sum of the fluxes based on the day of the week and the province
def flow_days(data_province_selected, cod_pro_selected):
    flow_days=data_province_selected.groupby(['DOW', 'COD_PRO']).FLOW.sum().reset_index()
    return flow_days

#Define a dictionary with a dataframe 
d_flow_days = {} 

for i in range(len(reg)):
    d_flow_days[reg[i]] = pd.DataFrame(flow_days(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))


#Printing the dataframe associating the name to each province code
for i in range(len(reg)):
    name_list=[]
    for cod in d_flow_days[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] #torna poco questo 00
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_list.append(name)  
    d_flow_days[reg[i]].insert(1, "NAME", name_list)


print("\n")

#Analysis based on the days
days = ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì', 'Sabato', 'Domenica']

#Function to order the dataset based on the days of the week 
def days_order(dataset,days):
    dataset['DOW']=pd.Categorical(dataset['DOW'], categories=days, ordered=True)
    dataset_days=dataset.sort_values(['DOW'], ignore_index=True)
    return dataset_days

#Function to sort the data by provinces
def sort(dataset):
    dataset=dataset.sort_values(['COD_PRO', 'NAME'],ascending=False, ignore_index=True)
    return dataset

#Define a dictionary cointaing a dataframe for each regions 
#Different dataframes based on different type of data organization
flow_each_pro={}

for i in range(len(reg)):
    list_k=[0]
    flow_each_pro[reg[i]]=sort(d_flow_days[reg[i]])
    for k in range(1,len(flow_each_pro[reg[i]]['COD_PRO'])):
        if flow_each_pro[reg[i]]['COD_PRO'][k]!=flow_each_pro[reg[i]]['COD_PRO'][k-1]:
            list_k.append(k)
    list_k.append(len(flow_each_pro[reg[i]]['COD_PRO']))
    for d in range(len(list_k)-1): 
            flow_each_pro[reg[i],d]= pd.DataFrame(days_order(flow_each_pro[reg[i]][list_k[d]:list_k[d+1]],days))
    flow_each_pro[reg[i]]=pd.DataFrame(days_order(d_flow_days[reg[i]],days))
    
    
#Funtion to sum the fluxes of a day for each region 
def flux_sum_day(flow_each_pro):
    flux_sum_pd=flow_each_pro.groupby(['DOW'])['FLOW'].sum().reset_index()
    return flux_sum_pd

#Dictionary 
flux_sum_perday={}

#Dataframe for each region with the sum of fluxes based on the day of the week
for i in range(len(reg)):
        flux_sum_perday[reg[i]]=flux_sum_day(flow_each_pro[reg[i]])
        
#DISPLAY THE RESULTS 

import plotly.graph_objects as go
from plotly.offline import iplot

#Results for the single regions based on the days of the week

#plot
fig3 = go.Figure()
color_dict = ['#b32d00', '#fb9804' ,'#b5de2b']
for i in range(len(reg)):  
        fig3 = fig3.add_trace(go.Scatter(x = flux_sum_perday[reg[i]]["DOW"], y = flux_sum_perday[reg[i]]["FLOW"],name =reg[i], marker=dict(color=color_dict[i],size=8,line=dict(color=color_dict[i],width=8))))
        fig3.update_layout(title={'text': "Flow for the selected regions based on the days of the week",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Day of the week'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})
fig3.show()

#dataset
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]   
for i in range(len(reg)):  
    fig_tab = ff.create_table(flux_sum_perday[reg[i]],colorscale=table_color)
    #fig_tab.update_layout(title={'text': str(reg[i]),'xanchor': 'left',yanchor': 'top'})
    fig_tab.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .8]}, yaxis1 = {'domain': [0., 1]})
    fig_tab.show()
    print("\n\n")
    
    
#Results for each province based on the days of the week
  
#plot
fig2 = go.Figure()
#color_dict = ['#b32d00', '#fb9804' ,'#b5de2b']

color_dict = [ ['#e61100','#ff1a1a','#ff9f80'], ['#ffff66','#ff8000', '#ffd480'], [ '#b5de2b','#4dff4d','#00cc00' ]]

for i in range(len(reg)):  
    for k in range(len(flow_pro[reg[i]]['COD_PRO'])):
        fig2 = fig2.add_trace(go.Bar(x = flow_each_pro[reg[i],k]["DOW"], y = flow_each_pro[reg[i],k]["FLOW"], name = flow_each_pro[reg[i],k].iloc[0,1], marker_color=color_dict[i][2-k]))
        fig2.update_layout(title={'text': "Flow for the mostly contributing provinces based on the days of the week",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},xaxis = {'domain': [0.1, 1.], 'title':'Day of the week'}, yaxis = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'})
fig2.show()

#Dataset
table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
    
for i in range(len(reg)):  
    fig_tab = ff.create_table(flow_each_pro[reg[i]],colorscale=table_color, name=reg[i])
    fig_tab.update_layout(title={'text': str(reg[i]),'xanchor': 'left','yanchor': 'top'}, xaxis1 = {'domain': [0.3, .8]}, yaxis1 = {'domain': [0., 1]})
    fig_tab.show()
    print("\n\n")

## 3. MORE PROVINCES FOR THE THREE HIGHWAYS 

In [447]:
##############################PROVA CITTA' autostrada

#For the regions of interest insert the region capital names and the region names
m_cap=['Milano', 'Bologna', 'Trieste', 'Roma', 'Torino', 'Firenze', 'Napoli', 'Venezia']
m_reg=['Lombardia', 'Emilia Romagna', 'Friuli Venezia Giulia', 'Lazio', 'Piemonte', 'Toscana', 'Campania', 'Veneto']

#Function to obtain the info of the regions of interest 
def info_regions(codici_istat_provincia, capoluogo):
    cod_reg=codici_istat_provincia[codici_istat_provincia['PROVINCIA'] == capoluogo]['COD_REG'].values[0] 
    info_reg=codici_istat_provincia[codici_istat_provincia['COD_REG'] == cod_reg]
    return info_reg

#Create a dictionary of dataframes for the different regions (cod info only)
m_d = {} 
for c in m_cap:
    m_d[m_reg[m_cap.index(c)]] = pd.DataFrame(info_regions(codici_istat_provincia,c))

#Define a dictionary with the dataframe for the different regions with all the info from data_day_od
#data_reg[reg[i]]  MONTH|DOW|ORIGIN|DESTINATION|CUST_CLASS|COD_CONTRY|COD_PRO|PRO_COM|FLOW
m_data_reg = {} 
for i in range(len(m_reg)):
    m_data_reg[m_reg[i]] = pd.DataFrame(data_regions(data_day_od,m_d[m_reg[i]]['COD_PRO'].values[:]))
    

code_padua=codici_istat_provincia[codici_istat_provincia['PROVINCIA']=='Padova']['COD_PRO'].values[0]

    
#######4.1: STUDY OF THE FLUXES OF NEARBY REGIONS TO FIND WHICH OF THE THREE DIRECTIONS HAS TO BE PRIORITIZED######
   
#Create a dictionary for the the total flux for the different regions 
m_d_flow = {}
for i in range(len(m_reg)):
    m_d_flow[m_reg[i]] = flow_regions(m_data_reg[m_reg[i]], m_d[m_reg[i]]['COD_PRO'].values[:])
    
    
#Dataframe with the total flux for the regions of interest, flow: REGION|FLOW|
m_flow= pd.DataFrame(m_d_flow.items(), columns=['REGION', 'FLOW']).sort_values(by='FLOW', ascending=False)

########## 4.2: STUDY OF THE FLUXES OF THE MOSTLY CONTRIBUTING PROVINCES BASED ON THE DAY OF THE WEEK #########

#SELECTION OF THE PROVINCES THAT ARE MOSTLY CONTRIBUTING BASED ON THE FLOW 

#Functions that finds the mostly contributing provinces for each region:
#Returns the total flux for each province and the code of the selected provinces
def flow_province(data_regions):
    flow_province=data_regions.groupby('COD_PRO').FLOW.sum()
    flow_sorted=flow_province.sort_values(ascending=False)
    flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index() #Selection based on the flux
    for cod in flow_sorted.reset_index()['COD_PRO'].values[:]:
        if cod == code_padua:
            flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index()
            flow_selected=flow_selected.loc[1:]
    cod_pro_selected=flow_selected['COD_PRO'].values[:]
    return flow_selected, cod_pro_selected


m_d_flow_province = {} #for function output
m_cod_pro_selected={} #for function output
m_flow_pro={} #to built a dictionary with a flow dataframe for the different regions
m_pro_selected={}  #to built a dictionary with the code of the selected provinces

for i in range(len(m_reg)): 
    m_d_flow_province, m_cod_pro_selected = flow_province(m_data_reg[m_reg[i]]) #function returns two different outputs
    m_flow_pro[m_reg[i]]= m_d_flow_province
    m_pro_selected[m_reg[i]]=m_cod_pro_selected
    

#Associating each province code the province name
#flow of the province for a region flow_pro[reg[i]]: 'COD PRO', 'NAME', 'FLOW'

for i in range(len(m_reg)):
    name_l=[]
    for cod in m_flow_pro[m_reg[i]]['COD_PRO'].values[:]:
        if cod in m_d[m_reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(m_d[m_reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] 
            name=m_d[m_reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_l.append(name)
    #print(reg[i])
    m_flow_pro[m_reg[i]].insert(1, "NAME", name_l)
    #print(flow_pro[reg[i]]) 
    
#Display the results
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px



fig5 = go.Figure()
color_dict=['#b32d00', '#fb9804' ,'#b5de2b','#6ece8c', '#9ce2bf','#94c0aa','#39604d','#2277a4']


#Plot
for i in range(len(m_reg)-1):  
        fig5.add_trace(go.Bar(x = m_flow_pro[m_reg[i]]["NAME"], y = m_flow_pro[m_reg[i]]["FLOW"], name=m_reg[i], marker_color=color_dict[i]))
        fig5.update_layout(barmode='relative')
        fig5.update_layout(margin=dict(l=80, r=180, t=90, b=90),)
fig5.show()

#With Veneto
fig_allp=go.Figure()
for i in range(len(m_reg)):  
        fig_allp.add_trace(go.Bar(x = m_flow_pro[m_reg[i]]["NAME"], y = m_flow_pro[m_reg[i]]["FLOW"], name=m_reg[i], marker_color=color_dict[i]))
        fig_allp.update_layout(barmode='relative')
        fig_allp.update_layout(margin=dict(l=80, r=180, t=90, b=90),)
fig_allp.show()

table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]

    

more_prov=pd.concat(m_flow_pro.values(), axis=0)
highway=['A4-west','A4-west','A4-west','A4-west','A13','A13','A4-east','A4-east','A13','A4-west','A13','A13',
         'A4-east','A4-west','A4-east','A4-west','A13','A4-east']
more_prov.insert(3,'DIR',highway)  
flow_dir=more_prov.groupby(by='DIR').FLOW.sum().reset_index().sort_values(by='FLOW',ascending=False)
#fig = ff.create_table(more_prov,colorscale=table_color)
#fig.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
#fig.show()

tab_flow_dir_prov= ff.create_table(more_prov,colorscale=table_color)
tab_flow_dir_prov.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
tab_flow_dir_prov.show()  

#Highways
color_dict=['#b5de2b','#b32d00', '#fb9804']
fig_flow_dir=go.Figure()
fig_flow_dir.add_trace(go.Bar(x = flow_dir["DIR"], y = flow_dir["FLOW"], marker_color=color_dict))
fig_flow_dir.update_layout(margin=dict(l=80, r=180, t=90, b=90),)
fig_flow_dir.show()


tab_flow_dir= ff.create_table(flow_dir,colorscale=table_color)
tab_flow_dir.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
tab_flow_dir.show()  

## Milocco's Part to add to the 6th part

#We made a cumulative bar plot for each direction and for the different months present in the data.
#As expect, the rank are a4-West, a4-East and a13-South mostly used in April and May.


In [445]:
#MESIIIII + MILOCCO

#Define a dictionary with a dataframe for the different regions, containg all the info of the selected provinces 
#print(d_selected_province)

def flow_month_reg(data_reg):
    flow_m_reg=data_reg.groupby(['MONTH']).FLOW.sum().reset_index()
    return flow_m_reg

d_flow_month_reg = {} 

for i in range(len(reg)):
    d_flow_month_reg[reg[i]] = pd.DataFrame(flow_month_reg(data_reg[reg[i]]))
    
def flow_month_pro(data_province_selected, cod_pro_selected):
    flow_m=data_province_selected.groupby(['MONTH', 'COD_PRO']).FLOW.sum().reset_index()
    return flow_m

d_flow_month_pro = {} 

for i in range(len(reg)):
    d_flow_month_pro[reg[i]] = pd.DataFrame(flow_month_pro(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))


#reconcatenate
df=d_flow_month_pro[reg[0]].merge(flow_pro[reg[0]], on="COD_PRO")
df["Regions"]=reg[0]
for i in range(1,len(reg)):
    df=pd.concat([df, d_flow_month_pro[reg[i]].merge(flow_pro[reg[i]], on="COD_PRO")], ignore_index=True)
    df=df.replace(np.nan, reg[i])
df.rename(columns={"FLOW_x":"FLOW", "NAME":"Province", "FLOW_y":"FLOW TOTAL"}, inplace=True)


df["MONTH"] = pd.Categorical(df["MONTH"],
                    categories=["Febbraio","Marzo","Aprile","Maggio"])
df.sort_values(by=["MONTH","FLOW"], inplace=True, ignore_index=True)
total=d_flow_month_pro["Lombardia"].FLOW.sum()


#Plot
color=['#b32d00', '#fb9804' ,'#b5de2b'][::-1]

import plotly.express as px
fig = px.bar(df, x = df["MONTH"], y = df["FLOW"], color=df["Regions"], color_discrete_sequence=color, text=df["Province"], title="Visitors of nearby regions on months")
#fig.update_xaxes(categoryorder="total ascending")
fig.show()

fig = px.bar(df, x = df["Regions"], y = df["FLOW"], color=df["MONTH"], color_discrete_sequence=px.colors.qualitative.D3, text=df["Province"], title="Visitors of nearby regions on months")
fig.update_xaxes(categoryorder="total ascending")
fig.show()


#Display the data
#tab_months= ff.create_table(df,colorscale=table_color)
#tab_months.update_layout(xaxis = {'domain': [.1, .9]}, yaxis1 = {'domain': [0, 1]})
#tab_months.show()  
